In [1]:
import time
from typing import Tuple, Callable, List
import numpy as np
import plotly.graph_objs as go
from scipy.linalg import solve_banded, solve

## Аналитическое решение
Аналитическим решением ОДУ $-u''(x) + u(x) = x, ~ 0 \le x \le 1, ~ u(0) = u(1) = 0$
является функция $u(x) = x - \frac{e^{1-x} - e^{x+1}}{1 - e^2}$.

Давайте его запрограммируем:

In [2]:
def get_grid(N: int) -> np.ndarray:
    h = 1 / (N + 1)
    return np.linspace(h, 1, num=N, endpoint=False, dtype=np.float64)


In [3]:
def u(grid: np.ndarray) -> np.ndarray:
    return grid - ((np.exp(1 - grid) - np.exp(grid + 1)) / (1 - np.exp(2)))

Далее построим матрицу $A$ и вектор $b$ такие,
что решение $Au = b$ давало значения функции $u$ в точках $x_1, \dots, x_N$.

In [4]:
def get_A_b(grid: np.ndarray, band: bool) -> Tuple[np.ndarray, np.ndarray]:
    N = grid.shape[0]
    h = 1 / (N + 1)
    if band:
        A = np.array([[-(h ** -2)] * N, [2 * (h ** -2) + 1] * N, [-(h ** -2)] * N])
    else:
        A = (
            np.eye(N, k=0, dtype=np.float64) * (2 * (h ** -2) + 1) +
            np.eye(N, k=-1, dtype=np.float64) * (-(h ** -2)) +
            np.eye(N, k=1, dtype=np.float64) * (-(h ** -2))
        )
    b = grid

    return A, b

### Решение с помощью встроенной функции
В качестве встроенной функции я использую `scipy.linalg.solve`.
Также я решил протестировать `scipy.linalg.solve_banded`,
так как этот метод умеет решать СЛАУ в ленточном виде.

In [5]:
def solve_scipy(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    return solve(A, b)

In [6]:
def solve_scipy_band(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    return solve_banded((1, 1), A, b)

### Решение методом прогонки

In [ ]:
def solve_thomas(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    b = b.copy()
    d0 = A[0].copy()
    d1 = A[1].copy()
    d2 = A[2].copy()
    num_eqs = b.shape[0]
    for i in range(1, num_eqs):
        m = d2[i - 1] / d1[i - 1]
        d1[i] -= m * d0[i - 1]
        b[i] -= m * b[i - 1]

    x = d1
    x[-1] = b[-1] / d1[-1]

    for i in reversed(range(num_eqs - 1)):
        x[i] = (b[i] - d0[i] * x[i + 1]) / d1[i]

    return x

### Анализ работы встроенных и реализованной функций

In [7]:
def calc_error(predicted: np.array, true: np.array) -> np.ndarray:
    return np.linalg.norm(predicted - true)

In [13]:
def analyze(solvers: List[Tuple[Callable, bool]],
            true_func: Callable[[np.ndarray], np.ndarray],
            max_N: int, num_N: int, iters_to_time: int) -> None:

    times = [[] for _ in range(len(solvers))]
    errors = [[] for _ in range(len(solvers))]

    Ns = np.linspace(50, max_N, num=num_N, endpoint=True, dtype=np.int64)

    for N in Ns:
        grid = get_grid(N)
        true = true_func(grid)

        for i, solver in enumerate(solvers):
            # Error calculation
            solver, banded = solver
            A, b = get_A_b(grid, band=banded)
            predicted = solver(A, b)
            error = calc_error(predicted, true)
            errors[i].append(error)

            # Time measure
            local_times = []
            for _ in range(iters_to_time):
                start = time.time()
                _ = solver(A, b)
                local_times.append(time.time() - start)
            times[i].append(np.array(local_times[int(iters_to_time / 10):]).mean())

    fig_error = go.Figure()
    for e, solver in zip(errors, solvers):
        name = solver[0].__name__
        print(f"Building error plot for {name}")
        fig_error.add_trace(
            go.Scatter(x=Ns, y=e, mode='lines+markers', name=f"{name}")
        )
    fig_error.update_layout(
        yaxis={"title": "Error", "exponentformat": "e", "type": "log"},
        xaxis={"title": "N"},
        title=f"Error dependency by N"
    )
    fig_error.show()

    fig_time = go.Figure()
    for t, solver in zip(times, solvers):
        name = solver[0].__name__
        fig_time.add_trace(
            go.Scatter(x=Ns, y=t, mode='lines+markers', name=f"{name}")
        )
    fig_time.update_layout(
        yaxis={"title": "Time"},
        xaxis={"title": "N"},
        title=f"Time dependency by N"
    )
    fig_time.show()


In [14]:
analyze([(solve_scipy, False), (solve_thomas, True), (solve_scipy_band, True)],
        u, max_N=1000, num_N=50, iters_to_time=100)

Building error plot for solve_scipy
Building error plot for solve_thomas
Building error plot for solve_scipy_band


Как видно из графиков, точность всех методов идентична, что немного меня настораживает, 
но всё же ошибки в построении графиков нет, как видно из вывода ячейки.

Время работы встроенного решения `scipy.linalg.solve` растёт гораздо быстрее,
чем методы прогонки. Хотя конечно константа у `scipy.linalg.solve_banded`
гораздо меньше, чем у моего решения.

